In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/data_package_for_agesexcls.zip" -d "/content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/resized_images/F0742_IND_S_16_-45_01.JPG  
  inflating: /content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/resized_images/F0742_IND_S_16_-45_02.JPG  
  inflating: /content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/resized_images/F0742_IND_S_16_-90_01.JPG  
  inflating: /content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/resized_images/F0742_IND_S_16_-90_02.JPG  
  inflating: /content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/resized_images/F0742_IND_S_16_0_01.JPG  
  inflating: /content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/resized_images/F0742_IND_S_16_0_02.JPG  
  inflating: /content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/resized_images/F0742_IND_S_16_45_01.JPG  
  inflating: /content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/resized_images/F0742_IND_S_1

# Library Import

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os

# 데이터 로드

In [ ]:
DATA_DIR="/content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/resized_images"
LABEL_CSV="/content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/ALL_labels.csv"

In [ ]:
df=pd.read_csv(LABEL_CSV)
df.head()

,filename,male,female,0s,10s,20s,30s,40s,50s,60+
0,F0001_IND_D_18_-45_01.JPG,0,1,0,1,0,0,0,0,0
1,F0001_IND_D_18_-45_02.JPG,0,1,0,1,0,0,0,0,0
2,F0001_IND_D_18_-45_03.JPG,0,1,0,1,0,0,0,0,0
3,F0001_IND_D_18_-45_04.JPG,0,1,0,1,0,0,0,0,0
4,F0001_IND_D_18_-90_01.JPG,0,1,0,1,0,0,0,0,0


# 이미지 제너레이터 설정

In [ ]:
train_df, test_df=train_test_split(df, test_size=0.2, random_state=42)  # train/test
train_df, val_df=train_test_split(train_df, test_size=0.2, random_state=42)  # train/val

In [ ]:
IMAGE_SIZE=(224, 224)
BATCH_SIZE=16
columns=["male", "female", "0s", "10s", "20s", "30s", "40s", "50s", "60+"]

train_gen=ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.3, 1.0],
    zoom_range=0.3
)
val_test_gen=ImageDataGenerator(rescale=1./255)

> ## class_mode 설명:  
- 'categorical': One-hot 인코딩된 단일 클래스 -> 다중 클래스 분류 (예: 고양이/개/말)  
- 'binary': 0 또는 1 -> 이진 분류 (예: 남/여)  
- 'sparse': 정수 라벨 (0, 1, 2...) -> 다중 클래스 분류 (정수형 라벨)  
- 'input': 입력과 출력이 같은 경우 (예: 오토인코더)  
- None: 라벨 없이 X만 반환 (예측/테스트용)  
- 'raw': 다중 열을 벡터로 그대로 사용 -> 멀티레이블 분류 또는 회귀  
>
> 📌 최종 선택:  
멀티레이블 문제이므로 'raw' 사용  
-> 출력층: Dense(9, activation='sigmoid')  
-> 손실 함수: binary_crossentropy  
-> 평가 지표(metrics): accuracy 등  

In [ ]:
# train_genrator 생성
train_generator=train_gen.flow_from_dataframe(
    dataframe=train_df,
    directory=DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    x_col="filename",
    y_col=columns,
    class_mode="raw",
    shuffle=True,
    seed=42
)

Found 28253 validated image filenames.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 3 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
# val(val_test_genrator 생성)
val_generator=val_test_gen.flow_from_dataframe(
    dataframe=val_df,
    directory=DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    x_col="filename",
    y_col=columns,
    class_mode="raw",
    shuffle=False,
    seed=42
)

Found 7064 validated image filenames.


In [ ]:
# test(val_test_genrator 생성)
test_generator=val_test_gen.flow_from_dataframe(
    dataframe=test_df,
    directory=DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    x_col="filename",
    y_col=columns,
    class_mode="raw",
    shuffle=False,
    seed=42
)

Found 8831 validated image filenames.


# 모델 정의
> EfficientNetV2B0

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
basemodel=EfficientNetV2B0(include_top=False, weights="imagenet", input_shape=(224,224,3))

# 파인튜닝을 위해 상위 레이어는 훈련 가능하게 설정 (혹은 원하는 만큼 freeze 가능)
basemodel.trainable=True
basemodel.summary()

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "efficientnetv2-b0"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          0 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ normalization[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      4,608 │ stem_activation[… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 112, 112,  │          0 │ block1a_project_… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 56, 56,    │      9,216 │ block1a_project_… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 56, 56,    │        256 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 56, 56,    │          0 │ block2a_expand_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_co… │ (None, 56, 56,    │      2,048 │ block2a_expand_a… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_bn  │ (None, 56, 56,    │        128 │ block2a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_conv │ (None, 56, 56,    │     36,864 │ block2a_project_… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_bn   │ (None, 56, 56,    │        512 │ block2b_expand_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_act… │ (None, 56, 56,    │          0 │ block2b_expand_b

 Total params: 5,919,312 (22.58 MB)

 Trainable params: 5,858,704 (22.35 MB)

 Non-trainable params: 60,608 (236.75 KB)

In [ ]:
inputs=Input(shape=(224,224,3))
x=basemodel(inputs, training=True)
x=GlobalAveragePooling2D()(x)
outputs=Dense(len(columns), activation="sigmoid")(x)  # 다중 라벨 분류

model=Model(inputs, outputs)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),  # default: 0.001
              loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 9)              │        11,529 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,930,841 (22.62 MB)

 Trainable params: 5,870,233 (22.39 MB)

 Non-trainable params: 60,608 (236.75 KB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

esc=EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True, verbose=1)
model_checkpoint=ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/effmodel_best_model.h5",
                                 monitor="val_loss", mode="min", save_best_only=True, verbose=1)
#reducelr=ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[esc, model_checkpoint]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1766/1766 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8435 - loss: 0.2577
Epoch 1: val_loss improved from inf to 0.75140, saving model to /content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/effmodel_best_model.h5


1766/1766 ━━━━━━━━━━━━━━━━━━━━ 10978s 6s/step - accuracy: 0.8435 - loss: 0.2576 - val_accuracy: 0.0422 - val_loss: 0.7514
Epoch 2/10
1520/1766 ━━━━━━━━━━━━━━━━━━━━ 1:08 277ms/step - accuracy: 0.9264 - loss: 0.1603

In [ ]:
from tensorflow.keras.models import load_model
model=load_model("/content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/effmodel_best_model.h5")

model.evaluate(test_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


552/552 ━━━━━━━━━━━━━━━━━━━━ 1680s 3s/step - accuracy: 0.0390 - loss: 0.7520


[0.7511643171310425, 0.04212433472275734]

In [ ]:
# 저장할 경로 (네 드라이브 내 경로로 수정해도 OK)
model_save_path = "/content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/efficientnetv2_model.h5"

# 모델 저장 (훈련 종료 후 자동 저장)
model.save(model_save_path)
print(f"모델 저장 완료: {model_save_path}")

In [ ]:
# test 평가
test_loss, test_acc = model.evaluate(test_generator)
print(f"테스트 결과: Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# 정확도 시각화
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

# 손실 시각화
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

NameError: name 'history' is not defined

In [ ]:
log_df = pd.DataFrame(history.history)
log_path = "/content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/train_log.csv"
log_df.to_csv(log_path, index=False)
print(f"학습 로그 저장 완료: {log_path}")

In [ ]:
# from tensorflow.keras.models import load_model

# # 저장한 전체 모델 불러오기
# model = load_model("/content/drive/MyDrive/Colab Notebooks/vision_project/re_agesexcls/efficientnetv2_model.h5")

# # 이어서 재학습 가능
# model.fit(...)